<a href="https://colab.research.google.com/github/jhudsy/REF2021/blob/main/REF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openpyxl
!pip install jupyter-dash
!wget -O REF.xlsx https://results2021.ref.ac.uk/profiles/export-all

import numpy as np
import pandas as pd
import ipywidgets as widgets
from ipywidgets import interact, interact_manual,fixed,Layout
import plotly.express as px

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.9 MB 26.0 MB/s 
     |████████████████████████████████| 357 kB 65.1 MB/s 
     |████████████████████████████████| 1.6 MB 55.9 MB/s 
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11447 sha256=55e096cb869ba808d36f0a5bfd8033844c7588876c612364341ca986fc56ea99
  Stored in directory: /root/.cache/pip/wheels/f9/8d/8d/f6af3f7f9eea3553bc2fe6d53e4b287dad18b06a861ac56ddf
Successfully built retrying
--2022-09-02 09:16:46--  https://results2021.ref.ac.uk/profiles/export-all
Resolving results2021.ref.ac.uk (results2021.ref.ac.uk)... 13.107.246.40, 13.107.213.40, 2620:1ec:bdf::40, ...
Connecting to results2021.ref.ac.uk (results2021.ref.ac.uk)|13.107.246.40|:443... connected.
HTTP request sent, awaiting response... 200 OK
Cookie coming

In [2]:
df=pd.read_excel('REF.xlsx',header=6)
df=df.replace('-',0)
df[['4*','3*','2*','1*','Unclassified','Unit of assessment number']]=df[['4*','3*','2*','1*','Unclassified','Unit of assessment number']].apply(pd.to_numeric)
df=df.rename(columns={"Unclassified":"U*"}) #for ease of use

#let's add some columns for GPA and number of papers and position to speed up processing later
df['gpa']=df.apply(lambda r:(r['1*']*1+r['2*']*2+r['3*']*3+r['4*']*4)/100, axis=1)

df['tot_papers']=(0.5+(df[df['Profile']=='Outputs']['FTE of submitted staff']*2.5)).apply(np.floor)
df['n4*']=df[df['Profile']=='Outputs'].apply(lambda r:np.round(r['4*']*r['tot_papers']/100),axis=1)
df['n3*']=df[df['Profile']=='Outputs'].apply(lambda r:np.round(r['3*']*r['tot_papers']/100),axis=1)
df['n2*']=df[df['Profile']=='Outputs'].apply(lambda r:np.round(r['2*']*r['tot_papers']/100),axis=1)
df['n1*']=df[df['Profile']=='Outputs'].apply(lambda r:np.round(r['1*']*r['tot_papers']/100),axis=1)
df['nU*']=df[df['Profile']=='Outputs'].apply(lambda r:np.round(r['U*']*r['tot_papers']/100),axis=1)

df['position']=df.apply(lambda r: 1+df[(df['Unit of assessment number']==r['Unit of assessment number']) &
                                           (df['Profile']==r['Profile']) &
                                           (df['gpa']>float(r['gpa']))]['gpa'].count(),axis=1)

odf=df.copy()


In [34]:
def color_quartile(r):
    col=['green','yellow','orange','red'] 
    return [f"background-color: {col[r['quartile']-1]}"]*len(r)


def get_summary(df,institution,uoa):
    tot=len(df[df['Unit of assessment number']==uoa]['Institution name'].unique())
    mask=(df['Institution name']==institution) & (df['Unit of assessment number']==uoa)
    o=df[mask][['Profile','position','gpa']]
    o['quartile']=o.apply(lambda r: 4 if int(1+4.0*r['position']/tot)==5 else int(1+4.0*r['position']/tot),axis=1) #hack alert to do quartile
    print("N:",tot)
    
    return o.style.apply(color_quartile,axis=1)

def draw_ecdf(df,institution,uoa,profile):

    mask=(df['Profile']==profile) & (df['Unit of assessment number']==uoa)
    d=df[mask][['gpa','Institution name']]
    fig=px.ecdf(d,x='gpa',ecdfnorm=None,markers=True,lines=False,hover_data=["Institution name","gpa"])
    fig.add_vline(x=float(df[mask & (df['Institution name']==institution)]['gpa']))

    fig.add_vrect(x0=d['gpa'].quantile(q=0,interpolation='nearest'),
              x1=d['gpa'].quantile(q=0.25,interpolation='nearest'),
              line_width=0,fillcolor='red',opacity=0.2)
    fig.add_vrect(x0=d['gpa'].quantile(q=0.25,interpolation='nearest'),
              x1=d['gpa'].quantile(q=0.5,interpolation='nearest'),
              line_width=0,fillcolor='orange',opacity=0.2)
    fig.add_vrect(x0=d['gpa'].quantile(q=0.5,interpolation='nearest'),
              x1=d['gpa'].quantile(q=0.75,interpolation='nearest'),
              line_width=0,fillcolor='yellow',opacity=0.2)
    fig.add_vrect(x0=d['gpa'].quantile(q=0.75,interpolation='nearest'),
              x1=d['gpa'].quantile(q=1,interpolation='nearest'),
              line_width=0,fillcolor='green',opacity=0.2)
    return fig

def draw_gpa_vs_size(df,institution,uoa,profile):
  mask=(df['Profile']==profile) & (df['Unit of assessment number']==uoa)
  d=df[mask].sort_values('gpa')
  d=d.reset_index(drop=True)
  k=(d[d['Institution name']==institution].index[0])
  fig=px.scatter(d,x='FTE of submitted staff',y='gpa',hover_data=["Institution name","gpa","FTE of submitted staff"])
  n=len(d)
  trace=next(fig.select_traces())
  color = [trace.marker.color] * n
  color[k] = "red"
  trace.marker.color = color
  return fig

In [35]:
from numpy.core.fromnumeric import size
#define widgets
uoa_d=widgets.Dropdown(options=df['Unit of assessment number'].sort_values().unique(),description="UoA",layout=Layout(width='150px'))
inst_d=widgets.Dropdown(options=df[df['Unit of assessment number']==uoa_d.value]['Institution name'].sort_values().unique(),description="Institution")
profile_d=widgets.Dropdown(options=['Overall','Outputs','Environment','Impact'],description="Profile",layout=Layout(width='200px'))
outputs_t={}
impact_t={}
environment_t={}
for i in ['4*','3*','2*','1*','U*']:
    outputs_t[i]=widgets.Text(value=str(int(df[(df['Unit of assessment number']==uoa_d.value) & (df['Institution name']==inst_d.value) & (df['Profile']=='Outputs')][f"n{i}"])),description=i,layout=Layout(width='150px'))
    impact_t[i]=widgets.Text(value=str(float(df[(df['Unit of assessment number']==uoa_d.value) & (df['Institution name']==inst_d.value) & (df['Profile']=='Impact')][i])),description=i,layout=Layout(width='150px'))
    environment_t[i]=widgets.Text(value=str(float(df[(df['Unit of assessment number']==uoa_d.value) & (df['Institution name']==inst_d.value) & (df['Profile']=='Environment')][i])),description=i,layout=Layout(width='150px'))

update_b=widgets.Button(description='Update')
reset_b=widgets.Button(description='Reset')

summary_output=widgets.Output()
hist_output=widgets.Output()
size_vs_gpa_output=widgets.Output()
error_output=widgets.Output(layout={'text-color':'red'})


################################################################
#link widgets to each other

def uoa_changed(change):
    inst_d.options=df[df['Unit of assessment number']==uoa_d.value]['Institution name'].sort_values().unique()

def inst_changed(change):
    uoa=uoa_d.value
    inst=inst_d.value
    for i in ['4*','3*','2*','1*','U*']:
        outputs_t[i].value=str(int(df[(df['Unit of assessment number']==uoa) &
                              (df['Institution name']==inst) &
                              (df['Profile']=='Outputs')][f"n{i}"]))
        impact_t[i].value=str(float(df[(df['Unit of assessment number']==uoa) &
                              (df['Institution name']==inst) &
                              (df['Profile']=='Impact')][i]))        
        environment_t[i].value=str(float(df[(df['Unit of assessment number']==uoa) &
                              (df['Institution name']==inst) &
                              (df['Profile']=='Environment')][i]))
    refresh_summary()
    refresh_hist()

def profile_changed(change):
    refresh_hist()

def update_pressed(button):
    uoa=uoa_d.value
    inst=inst_d.value

    #sanity check that there are the right number of papers, else fail transparently
    s=0
    for i in ['4*','3*','2*','1*','U*']:
        s+=int(outputs_t[i].value)
    if s!=int(df[(df['Institution name']==inst) & (df['Unit of assessment number']==uoa) & (df['Profile']=='Outputs')]['tot_papers']): 
        refresh_error("Incorrect number of papers")
        return


    for i in ['4*','3*','2*','1*','U*']:
        df.loc[(df['Institution name']==inst) & (df['Unit of assessment number']==uoa) & (df['Profile']=='Environment'),i]=float(environment_t[i].value)
        df.loc[(df['Institution name']==inst) & (df['Unit of assessment number']==uoa) & (df['Profile']=='Impact'),i]=float(impact_t[i].value)
        df.loc[(df['Institution name']==inst) & (df['Unit of assessment number']==uoa) & (df['Profile']=='Outputs'),f"n{i}"]=int(outputs_t[i].value)
        outputs_percent=100*float(outputs_t[i].value)/float(df[(df['Institution name']==inst) &
                                                               (df['Unit of assessment number']==uoa) &
                                                               (df['Profile']=='Outputs')]['tot_papers'])
        df.loc[(df['Institution name']==inst) & (df['Unit of assessment number']==uoa) & (df['Profile']=='Outputs'),i]= outputs_percent
        df.loc[(df['Institution name']==inst) & (df['Unit of assessment number']==uoa) & (df['Profile']=='Overall'),i]=0.25*float(impact_t[i].value)+0.15*float(environment_t[i].value)+0.6*outputs_percent
        
    

    for p in ['Environment', 'Outputs', 'Impact','Overall']:
        gpa=0
        for i in range(1,5):
            gpa += i*float(df.loc[(df['Institution name']==inst) & (df['Unit of assessment number']==uoa) & (df['Profile']==p),f"{i}*"])
        gpa /= 100
        df.loc[(df['Institution name']==inst) & (df['Unit of assessment number']==uoa) & (df['Profile']==p),'gpa']=gpa

        df.loc[(df['Institution name']==inst) & (df['Unit of assessment number']==uoa) & (df['Profile']==p),'position']=1+df[(df['Unit of assessment number']==uoa) &
                                           (df['Profile']==p) &
                                           (df['gpa']>gpa)]['gpa'].count()  

    refresh_summary()
    refresh_hist()

def reset_pressed(button):
    global df,odf
    df=odf
    refresh_summary()
    refresh_hist()
        
##################################################################
#rendering functions
def refresh_error(error):
    error_output.clear_output()
    with error_output:
        display(error)

def refresh_summary():
    error_output.clear_output()
    summary_output.clear_output()
    with summary_output:
        display(get_summary(df,inst_d.value,uoa_d.value))
    
def refresh_hist():
    hist_output.clear_output()
    size_vs_gpa_output.clear_output()
    with hist_output:
        fig=draw_ecdf(df,institution=inst_d.value,uoa=uoa_d.value,profile=profile_d.value)
        fig.show()
    with size_vs_gpa_output:
        fig=draw_gpa_vs_size(df,institution=inst_d.value,uoa=uoa_d.value,profile=profile_d.value)
        fig.show()
        
uoa_d.observe(uoa_changed)
inst_d.observe(inst_changed)
profile_d.observe(profile_changed)
update_b.on_click(update_pressed)
reset_b.on_click(reset_pressed)

##################################################################
#And draw everything

display(widgets.HBox([uoa_d,inst_d,profile_d]))

grid=[widgets.VBox([widgets.Label(value="Outputs:"),widgets.Label(value="Impact:"),widgets.Label(value="Environment:",style = {'description_width': 'initial'})])]

for i in ['4*','3*','2*','1*','U*']:
    grid.append(widgets.VBox([outputs_t[i],impact_t[i],environment_t[i]]))
    
display(widgets.HBox(grid))
display(widgets.HBox([update_b,reset_b]))
display(error_output)
display(summary_output)
display(hist_output)
display(size_vs_gpa_output)

Output()

Output()

Output()

Output()